Some of these boxes are heavy duty (for the pc) and some of these are just light enough

In [2]:
# machine translation; USE THE PC FOR THIS
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-tl")

print(pipe("This is a job for me."))

Device set to use cpu


[{'translation_text': "Ito'y isang trabaho para sa akin."}]


In [14]:
import tika
from tika import parser
import os

print(os.listdir("./booksEpub"))

epubList = []

for i in os.listdir("./booksEpub"):
    filePath = "./booksEpub/" + i
    epubList.append(parser.from_file(filePath))

['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Pogi,_Si_Pogs_At_Ang_Tora-tora___Filipino-v12023.11.14T082955.epub', 'Super_Wanda___Filipino-v12023.04.07T025901.epub']


In [ ]:
import pymupdf
import os

print(os.listdir("./booksEpub"))

ebookContent = {}

for i in os.listdir("./booksEpub"):
    ebookText = ""
    titlePageText = ""
    filePath = "./booksEpub/" + i
    doc = pymupdf.open(filePath)
    for chapter in range(doc.chapter_count):
        print(f"Chapter {chapter}")
        if chapter==0:
            # get the texts and it will be the key
            currentPage = doc[(0,0)]
            titlePageText = currentPage.get_text()
        else:
            for page in range(doc.chapter_page_count(chapter)):
                print(f"page {page}")
                currentPage = doc[(chapter, page)]
                currentText = currentPage.get_text()
                print(currentText)
                # check if it's the last few pages
                if not(("Brought to you by" in currentText) or ("International License" in currentText) or ("Table of Contents" in currentText) or ("This book was developed as part of the ABC" in currentText) or ("Advancing Basic Education in the Philippines" in currentText)):
                    ebookText+=currentText
                else:
                    pass
    
    #print(ebookText)
    ebookContent[titlePageText] = ebookText




In [53]:
import re

for key in ebookContent.keys():
    processedKey = re.sub(r'[\.]','',processedKey)
    processedKey = re.sub(r' ','_',processedKey)
    processedKey = re.sub(r'\?','',processedKey)
    f = open(f"./textData1_pymupdf/{processedKey}.txt","w",encoding='utf-8')
    #print(ebookContent[key])
    f.writelines(ebookContent[key])
    f.close()
    #print("DONE")

In [56]:
import os
import re

sourceFolder = "./textData1_pymupdf"
textNum = 1

# format the text correctly first
for i in os.listdir(sourceFolder):
    filePathForTokenizingText = f"{sourceFolder}/{i}"
    f = open(filePathForTokenizingText,'r', encoding='utf-8')
    insideText = f.read()
    f.close()

    #formatting with regex

    insideText = re.sub(r'[.]','. ',insideText)
    insideText = re.sub('\n',' ',insideText)
    insideText = re.sub('- ','-',insideText)
    insideText = re.sub('  ',' ',insideText)
    insideText = re.sub('&nbsp;','',insideText)
    

    f2 = open("./textData_forTokenization/text"+str(textNum), "w", encoding='utf-8')
    f2.writelines(insideText)
    f2.close()

    textNum+=1
    

making the vocabulary and the tokens

In [6]:
import os

postagged_filepath = "./textData_postagged/"

vocabulary = set()

for i in os.listdir(postagged_filepath):
    f = open(f"{postagged_filepath}{i}", "r", encoding="utf-16")
    story = f.read()
    
    for token in story.split():
        vocabulary.add(token)

In [9]:
import pandas as pd

vocab_df = pd.DataFrame(list(vocabulary), columns=['tokens'])
vocab_df['lemma'] = vocab_df['tokens'].apply(lambda x: x.split('|')[0])
vocab_df['postag'] = vocab_df['tokens'].apply(lambda x: x.split('|')[1])
vocab_df

,tokens,lemma,postag
0,baluktot|NNC,baluktot,NNC
1,naaamoy|VBTR,naaamoy,VBTR
2,Aray|NNP,Aray,NNP
3,nalulunod|VBTR,nalulunod,VBTR
4,nakatatakot|JJD,nakatatakot,JJD
...,...,...,...
1423,Umiiyak|VBTR_VBAF,Umiiyak,VBTR_VBAF
1424,natumba|VBTS,natumba,VBTS
1425,Mam|NNCA,Mam,NNCA
1426,naanod|VBTS,naanod,VBTS


In [15]:
pd.set_option('display.max_rows', None)
vocab_df.loc[ vocab_df['postag']=="NNC" ]['tokens']

0           baluktot|NNC
12             kulay|NNC
26        magkakampi|NNC
31            ibabaw|NNC
43               nun|NNC
56              mata|NNC
63              lola|NNC
66         halimbawa|NNC
67       paghihintay|NNC
69              bato|NNC
70           kalaban|NNC
79             dahan|NNC
94            bangin|NNC
98              tuwa|NNC
108          hapunan|NNC
113           sapsap|NNC
114          tagpiin|NNC
118             talo|NNC
121          pamilya|NNC
126             ulan|NNC
127            buhos|NNC
128           tutubi|NNC
142             sapa|NNC
148             anak|NNC
149          garapon|NNC
152            ugali|NNC
153           itsura|NNC
154        paliwanag|NNC
155           kwarto|NNC
156              ulo|NNC
166          managat|NNC
171           pataas|NNC
176            aaral|NNC
178           hantik|NNC
183            bagay|NNC
185             taon|NNC
187              Mam|NNC
189         kalayuan|NNC
195            hugis|NNC
203             Kuya|NNC
